<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/Haddop_nextflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hadoop

## Calcular el promedio

In [ ]:
%%writefile mapper1.py
#!/usr/bin/env python
import sys

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    number = float(line.strip())
    # Emite el número como clave y valor
    print("Total", number)
    print("Count", 1)

Writing mapper1.py


In [ ]:
!chmod 777 /content/mapper1.py /content/reducer1.py

chmod: cannot access '/content/reducer1.py': No such file or directory


In [ ]:
%%writefile reducer1.py
#!/usr/bin/env python
import sys

current_key = None
total = 0
count = 0

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    key, value = line.strip().split(" ")
    value = float(value)

    if key == "Total":
        total += value
    elif key == "Count":
        count += int(value)

# Calcula el promedio
average = total / count
print("Average", average)


Writing reducer1.py


In [ ]:
!seq 1 100 | python mapper1.py

Total 1.0
Count 1
Total 2.0
Count 1
Total 3.0
Count 1
Total 4.0
Count 1
Total 5.0
Count 1
Total 6.0
Count 1
Total 7.0
Count 1
Total 8.0
Count 1
Total 9.0
Count 1
Total 10.0
Count 1
Total 11.0
Count 1
Total 12.0
Count 1
Total 13.0
Count 1
Total 14.0
Count 1
Total 15.0
Count 1
Total 16.0
Count 1
Total 17.0
Count 1
Total 18.0
Count 1
Total 19.0
Count 1
Total 20.0
Count 1
Total 21.0
Count 1
Total 22.0
Count 1
Total 23.0
Count 1
Total 24.0
Count 1
Total 25.0
Count 1
Total 26.0
Count 1
Total 27.0
Count 1
Total 28.0
Count 1
Total 29.0
Count 1
Total 30.0
Count 1
Total 31.0
Count 1
Total 32.0
Count 1
Total 33.0
Count 1
Total 34.0
Count 1
Total 35.0
Count 1
Total 36.0
Count 1
Total 37.0
Count 1
Total 38.0
Count 1
Total 39.0
Count 1
Total 40.0
Count 1
Total 41.0
Count 1
Total 42.0
Count 1
Total 43.0
Count 1
Total 44.0
Count 1
Total 45.0
Count 1
Total 46.0
Count 1
Total 47.0
Count 1
Total 48.0
Count 1
Total 49.0
Count 1
Total 50.0
Count 1
Total 51.0
Count 1
Total 52.0
Count 1
Total 53.0
Count 1
To

In [ ]:
!seq 1  10000 | python mapper1.py | python reducer1.py

Average 5000.5


## ventas por producto

In [ ]:
%%writefile data.txt
Fecha,Producto,Cantidad,PrecioUnitario
2023-10-23,ProductoA,5,10.0
2023-10-23,ProductoB,3,15.0
2023-10-24,ProductoA,2,10.0
2023-10-24,ProductoC,1,20.0
2023-10-25,ProductoB,4,15.0
2023-10-25,ProductoC,2,20.0
2023-10-26,ProductoA,3,10.0
2023-10-26,ProductoB,2,15.0
2023-10-27,ProductoC,3,20.0
2023-10-27,ProductoD,1,25.0


Writing data.txt


In [ ]:
%%writefile mapper2.py

#!/usr/bin/env python
import sys

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    # Divide la línea en campos
    fields = line.strip().split(",")
    if len(fields) == 4:
        date, product, quantity, price = fields
        # Emite el producto como clave y la cantidad como valor
        print(f"{product}\t{quantity}")


Writing mapper2.py


In [ ]:
%%writefile reducer2.py

#!/usr/bin/env python
import sys

current_product = None
total_quantity = 0

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    product, quantity = line.strip().split("\t")
    quantity = int(quantity)

    if product == current_product:
        total_quantity += quantity
    else:
        if current_product:
            print(f"{current_product}\t{total_quantity}")
        current_product = product
        total_quantity = quantity

if current_product:
    print(f"{current_product}\t{total_quantity}")


Writing reducer2.py


In [ ]:
!cat data.txt | grep -v "Cantidad" |python mapper2.py | sort -k1,1 |python reducer2.py

ProductoA	10
ProductoB	9
ProductoC	6
ProductoD	1


## Puntajes de Peliculas.

In [ ]:
%%writefile movie_data.txt
UserID,MovieID,Rating,Timestamp
1,101,4,964982703
1,102,3,964982703
1,103,5,964982703
2,101,5,964982703
2,103,2,964982703
2,104,4,964982703
3,102,4,964982703
3,104,3,964982703
3,105,2,964982703
4,101,3,964982703
4,105,5,964982703
4,106,4,964982703
5,102,4,964982703
5,106,5,964982703
5,107,2,964982703
6,101,4,964982703
6,103,3,964982703
6,108,5,964982703
7,102,5,964982703
7,108,4,964982703
7,109,3,964982703
8,101,2,964982703
8,103,4,964982703
8,110,5,964982703
9,102,4,964982703
9,110,3,964982703
9,111,2,964982703
10,101,5,964982703
10,111,3,964982703
10,112,4,964982703
11,103,5,964982703
11,112,4,964982703
11,113,3,964982703
12,101,2,964982703
12,113,5,964982703
12,114,4,964982703
13,102,3,964982703
13,114,5,964982703
13,115,4,964982703
14,101,4,964982703
14,103,5,964982703
14,115,3,964982703
15,102,2,964982703
15,116,4,964982703
15,117,5,964982703
16,101,3,964982703
16,116,4,964982703
16,118,5,964982703
17,102,4,964982703
17,118,3,964982703
17,119,2,964982703
18,101,5,964982703
18,103,4,964982703
18,119,3,964982703
19,102,3,964982703
19,119,4,964982703
19,120,5,964982703
20,101,2,964982703
20,103,5,964982703
20,120,4,964982703
21,102,4,964982703
21,120,5,964982703
21,121,3,964982703
22,101,3,964982703
22,121,5,964982703
22,122,4,964982703
23,102,5,964982703
23,122,3,964982703
23,123,2,964982703
24,101,4,964982703
24,123,5,964982703
24,124,3,964982703
25,102,2,964982703
25,124,4,964982703
25,125,5,964982703

Writing movie_data.txt


In [ ]:
%%writefile mapper3.py

#!/usr/bin/env python
import sys

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    # Divide la línea en campos
    fields = line.strip().split(",")
    if len(fields) == 4:
        user, movie, rating, timestamp = fields
        # Emite el MovieID como clave y la calificación como valor
        print(f"{movie}\t{rating}")


Writing mapper3.py


In [ ]:
%%writefile reducer3.py
#!/usr/bin/env python
import sys

current_movie = None
total_rating = 0
total_reviews = 0

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    movie, rating = line.strip().split("\t")
    rating = int(rating)

    if movie == current_movie:
        total_rating += rating
        total_reviews += 1
    else:
        if current_movie:
            average_rating = total_rating / total_reviews
            print(f"{current_movie}\t{average_rating:.2f}")
        current_movie = movie
        total_rating = rating
        total_reviews = 1

if current_movie:
    average_rating = total_rating / total_reviews
    print(f"{current_movie}\t{average_rating:.2f}")


Writing reducer3.py


In [ ]:
!cat movie_data.txt  | python mapper3.py |grep -v "Movie" | sort -k1,1 | python reducer3.py | sort -nr -k2,2

125	5.00
117	5.00
120	4.67
114	4.50
108	4.50
106	4.50
103	4.12
121	4.00
118	4.00
116	4.00
113	4.00
112	4.00
110	4.00
102	3.58
101	3.54
124	3.50
123	3.50
122	3.50
115	3.50
105	3.50
104	3.50
119	3.00
109	3.00
111	2.50
107	2.00


## Multiplicacion de matrices con Hadoop

In [ ]:
!seq 1 100 | awk '{i++;print "A,"i","int(rand()*20); if(i==10){i=0}}' > matrix_in.txt
!seq 1 100 | awk '{i++;print "B,"i","int(rand()*20); if(i==10){i=0}}' >> matrix_in.txt
! head matrix_in.txt
! tail matrix_in.txt

A,1,10
A,2,14
A,3,7
A,4,13
A,5,6
A,6,0
A,7,2
A,8,7
A,9,1
A,10,4
B,1,5
B,2,7
B,3,5
B,4,14
B,5,14
B,6,12
B,7,2
B,8,17
B,9,11
B,10,11


# Nextflow

In [1]:
!java -version

openjdk version "11.0.20.1" 2023-08-24
OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [2]:
!curl -s https://get.nextflow.io | bash

CAPSULE: Downloading dependency org.multiverse:multiverse-core:jar:0.7.0
CAPSULE: Downloading dependency org.apache.ivy:ivy:jar:2.5.1
CAPSULE: Downloading dependency org.codehaus.groovy:groovy:jar:3.0.19
CAPSULE: Downloading dependency commons-io:commons-io:jar:2.11.0
CAPSULE: Downloading dependency com.google.guava:listenablefuture:jar:9999.0-empty-to-avoid-conflict-with-guava
CAPSULE: Downloading dependency io.nextflow:nf-httpfs:jar:23.10.0
CAPSULE: Downloading dependency com.beust:jcommander:jar:1.35
CAPSULE: Downloading dependency io.nextflow:nf-commons:jar:23.10.0
CAPSULE: Downloading dependency jline:jline:jar:2.9
CAPSULE: Downloading dependency org.codehaus.groovy:groovy-xml:jar:3.0.19
CAPSULE: Downloading dependency org.slf4j:log4j-over-slf4j:jar:2.0.7
CAPSULE: Downloading dependency com.github.zafarkhaja:java-semver:jar:0.9.0
CAPSULE: Downloading dependency javax.mail:mail:jar:1.4.7
CAPSULE: Downloading dependency com.fasterxml.jackson.core:jackson-annotations:jar:2.14.2
CAPSU

In [3]:
! /content/nextflow run hello

N E X T F L O W  ~  version 23.10.0
Pulling nextflow-io/hello ...
 downloaded from https://github.com/nextflow-io/hello.git
Launching `https://github.com/nextflow-io/hello` [gigantic_noyce] DSL2 - revision: 1d71f857bb [master]
[-        ] process > sayHello -

[-        ] process > sayHello [  0%] 0 of 3

executor >  local (4)
[e3/1e1ca3] process > sayHello (3) [ 75%] 3 of 4
Ciao world!

Bonjour world!

Hello world!


executor >  local (4)
[38/b2c8fe] process > sayHello (4) [100%] 4 of 4 ✔
Ciao world!

Bonjour world!

Hello world!

Hola world!




In [4]:
%%writefile sayhello.nf

nextflow.enable.dsl=2
process sayHello {
    input:
        val cheers

    output:
        stdout emit: verbiage

    script:
    """
    echo -n $cheers
    """
}

workflow {
    things = channel.of('Hola Mundo!', 'Hello world!', 'Bazinga!1')
    sayHello(things)
    sayHello.out.verbiage.view()
}

Writing sayhello.nf


In [5]:
! /content/nextflow run  sayhello.nf

N E X T F L O W  ~  version 23.10.0
Launching `sayhello.nf` [condescending_bardeen] DSL2 - revision: 502b7e396e
[-        ] process > sayHello -

executor >  local (2)
[6c/dfbf43] process > sayHello (2) [  0%] 0 of 3

executor >  local (3)
[52/00b24b] process > sayHello (3) [100%] 3 of 3 ✔
Hola Mundo!
Hello world!
Bazinga!1



## Singularity configuration

In [8]:
!add-apt-repository -y ppa:apptainer/ppa

Repository: 'deb https://ppa.launchpadcontent.net/apptainer/ppa/ubuntu/ jammy main'
Description:
This PPA contains stable releases of Apptainer.

Documentation https://apptainer.org/docs/
Support https://apptainer.org/help/
Bug report https://github.com/apptainer/apptainer/issues
More info: https://launchpad.net/~apptainer/+archive/ubuntu/ppa
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/apptainer-ubuntu-ppa-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/apptainer-ubuntu-ppa-jammy.list
Adding key to /etc/apt/trusted.gpg.d/apptainer-ubuntu-ppa.gpg with fingerprint F6B0F5193D4F3301EF491FF0AFE36534FC6218AE
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [556 kB]
Get:5 htt

In [9]:
!apt update

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/apptainer/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
22 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [10]:
! apt install -y apptainer

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fakeroot fuse-overlayfs fuse2fs fuse3 libfakeroot libfuse3-3 liblzo2-2 libsquashfuse0
  squashfs-tools squashfuse uidmap
The following packages will be REMOVED:
  fuse
The following NEW packages will be installed:
  apptainer fakeroot fuse-overlayfs fuse2fs fuse3 libfakeroot libfuse3-3 liblzo2-2 libsquashfuse0
  squashfs-tools squashfuse uidmap
0 upgraded, 12 newly installed, 1 to remove and 22 not upgraded.
Need to get 27.3 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfuse3-3 amd64 3.10.5-1build1 [81.2 kB]
Get:2 https://ppa.launchpadcontent.net/apptainer/ppa/ubuntu jammy/main amd64 apptainer amd64 1.2.4-1~ubuntu22.04.0+stable1 [26.8 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 fuse3 amd64 3.10.5-1build1 [24.7 kB]
Get:4 http:

In [11]:
! apt install -y apptainer-suid

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  apptainer-suid
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 5,600 kB of archives.
After this operation, 17.5 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/apptainer/ppa/ubuntu jammy/main amd64 apptainer-suid amd64 1.2.4-1~ubuntu22.04.0+stable1 [5,600 kB]
Fetched 5,600 kB in 2s (2,848 kB/s)
Selecting previously unselected package apptainer-suid.
(Reading database ... 122584 files and directories currently installed.)
Preparing to unpack .../apptainer-suid_1.2.4-1~ubuntu22.04.0+stable1_amd64.deb ...
Unpacking apptainer-suid (1.2.4-1~ubuntu22.04.0+stable1) ...
Setting up apptainer-suid (1.2.4-1~ubuntu22.04.0+stable1) ...


In [12]:
! singularity

Usage:
  apptainer [global options...] <command>

Available Commands:
  build       Build an Apptainer image
  cache       Manage the local cache
  capability  Manage Linux capabilities for users and groups
  checkpoint  Manage container checkpoint state (experimental)
  completion  Generate the autocompletion script for the specified shell
  config      Manage various apptainer configuration (root user only)
  delete      Deletes requested image from the library
  exec        Run a command within a container
  inspect     Show metadata for an image
  instance    Manage containers running as services
  key         Manage OpenPGP keys
  oci         Manage OCI containers
  overlay     Manage an EXT3 writable overlay image
  plugin      Manage Apptainer plugins
  pull        Pull an image from a URI
  push        Upload image to the provided URI
  remote      Manage apptainer remote endpoints, keyservers and OCI/Docker registry credentials
  run         Run the user-defined default comman

# Examples

In [24]:
! wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR134/021/SRR13447321/SRR13447321_1.fastq.gz
! wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR134/021/SRR13447321/SRR13447321_2.fastq.gz

--2023-10-25 14:33:35--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR134/021/SRR13447321/SRR13447321_1.fastq.gz
           => ‘SRR13447321_1.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.165|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/SRR134/021/SRR13447321 ... done.
==> SIZE SRR13447321_1.fastq.gz ... 693436099
==> PASV ... done.    ==> RETR SRR13447321_1.fastq.gz ... done.
Length: 693436099 (661M) (unauthoritative)

SRR13447321_1.fastq 100%[===================>] 661.31M  24.6MB/s    in 26s     

2023-10-25 14:34:04 (25.6 MB/s) - ‘SRR13447321_1.fastq.gz’ saved [693436099]

--2023-10-25 14:34:04--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR134/021/SRR13447321/SRR13447321_2.fastq.gz
           => ‘SRR13447321_2.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.sra.

In [29]:
%%writefile ejemplo1.nf

// Codigo nextflow
nextflow.enable.dsl=2
params.files=""
process NUM_LINEAS {
    input:
    path read

    script:
    """
    printf '${read} '
    gunzip -c ${read} | wc -l
    """
}

reads_ch = Channel.fromPath(params.files)

workflow {
  NUM_LINEAS(reads_ch)
}

Overwriting ejemplo1.nf


In [30]:
! /content/nextflow run  ejemplo1.nf --files "*.fastq.gz"  -process.echo

N E X T F L O W  ~  version 23.10.0
Launching `ejemplo1.nf` [magical_keller] DSL2 - revision: ab4c064c38
[-        ] process > NUM_LINEAS -

executor >  local (1)
[-        ] process > NUM_LINEAS [  0%] 0 of 2

executor >  local (2)
[10/833b31] process > NUM_LINEAS (2) [  0%] 0 of 2

executor >  local (2)
[10/833b31] process > NUM_LINEAS (2) [ 50%] 1 of 2
SRR13447321_1.fastq.gz 37769292


executor >  local (2)
[33/21ed7e] process > NUM_LINEAS (1) [100%] 2 of 2 ✔
SRR13447321_1.fastq.gz 37769292

SRR13447321_2.fastq.gz 37769292




In [31]:
%%writefile fastqc.nf

process FASTQC {
    tag "$sampleId-mem"
    //label 'process_medium'
    publishDir "$params.outdir/QC/FASTQC", mode: "copy"

  container "${ workflow.containerEngine == 'singularity' && !task.ext.singularity_pull_docker_container ?
        'https://depot.galaxyproject.org/singularity/fastqc:0.12.1--hdfd78af_0' :
        'biocontainers/fastqc:0.12.1--hdfd78af_0' }"

    input:
    tuple val(sampleId), val(part), file(read1), file(read2)

    output:
    path("${sampleId}-${part}.fastqc"), emit: fqc

    script:
    if(params.debug == true){
    """
    echo fastqc -o ${sampleId}-${part}.fastqc $read1 $read2
    mkdir -p ${sampleId}-${part}.fastqc
    touch ${sampleId}-${part}.fastqc/report.fastqc

    """
    } else{
    """
    mkdir -p ${sampleId}-${part}.fastqc
    fastqc -t $task.cpus -o ${sampleId}-${part}.fastqc $read1 $read2
    """
    }

}
workflow {
    if(params.reads != null){
    // --reads "./reads/B087*.merge.{1,2}.fq.gz"
    read_pairs_ch = channel.fromFilePairs(params.reads)
    }else if(params.csv != null){
    //we reads pairs from csv
    read_pairs_ch=Channel.fromPath(params.csv) \
        | splitCsv(header:true) \
        | map { row-> tuple(row.sampleId, row.part,  file(row.read1), file(row.read2)) }
    }else{
        println "Error: reads regex or path"
    }

    read_pairs_ch.view()
    FASTQC(read_pairs_ch)

}


Writing fastqc.nf


In [43]:
%%writefile input.csv
sampleId,part,read1,read2
test,0,/content/SRR13447321_1.fastq.gz,/content/SRR13447321_2.fastq.gz

Overwriting input.csv


In [45]:
! /content/nextflow run  fastqc.nf --csv "/content/input.csv"  -process.echo --debug true

N E X T F L O W  ~  version 23.10.0
Launching `fastqc.nf` [fabulous_swirles] DSL2 - revision: 2c67865c3e
WARN: Access to undefined parameter `reads` -- Initialise it to a default value eg. `params.reads = some_value`
[-        ] process > FASTQC -
[test, 0, /content/SRR13447321_1.fastq.gz, /content/SRR13447321_2.fastq.gz]

executor >  local (1)
[a1/ba98de] process > FASTQC (test-mem) [  0%] 0 of 1
[test, 0, /content/SRR13447321_1.fastq.gz, /content/SRR13447321_2.fastq.gz]

executor >  local (1)
[a1/ba98de] process > FASTQC (test-mem) [100%] 1 of 1 ✔
[test, 0, /content/SRR13447321_1.fastq.gz, /content/SRR13447321_2.fastq.gz]
fastqc -o test-0.fastqc SRR13447321_1.fastq.gz SRR13447321_2.fastq.gz

WARN: Access to undefined parameter `outdir` -- Initialise it to a default value eg. `params.outdir = some_value`
WARN: Process `FASTQC` publishDir path contains a variable with a null value

executor >  local (1)
[a1/ba98de] process > FASTQC (test-mem) [100%] 1 of 1 ✔
[test, 0, /content/SRR13447